# Compare ML Algorithms to Human
So far we've been comparing ML algorithms to "true" posteriors of each simulation dataset. Here we are comparing ML algorithms to human counterparts

In [1]:
# changing cwd
%cd ..

c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp


## Load packages

In [2]:
from src.inductive_bias import IB
ib = IB() #instantiate inductive bias package

[ c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp\clf\SimulationData.pickle ] loaded
[ c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp\clf\PosteriorData.pickle ] loaded
[ c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp\clf\TrainedCLF.pickle ] loaded
[ c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp\clf\EstimatedData.pickle ] loaded
[ c:\Users\jongm\Desktop\temp_workspace\Python\JOVO\induced_bias_exp\clf\HellingerData.pickle ] loaded
Size of the S-XOR: (6322,)                    
Size of the Spiral: (6276,)                    
Size of the whole dataset: (12598, 7)

Size of the S-XOR after adding hellinger: (6322, 8)            
Size of the Spiral after adding hellinger: (6276, 8)


## Load Dependencies

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle

cmap = 'PRGn'

import warnings
warnings.filterwarnings('ignore')

In [19]:
SAVEFIG = False

## Time and Date of the experiment

In [3]:
print(ib.date)

2021-08-09 07:40:46.841347


In [20]:
fig, axs = plt.subplots(1,4,figsize = (5*4,5))

for i in range(4):
    mlp = axs[i].scatter(ib.mask[:,0], ib.mask[:,1], c=mtype[0][i], cmap=cmap, vmin=0, vmax=1)
    axs[i].set_title(label[i], fontsize=18)
    axs[i].set_xlim([-3,3])
    axs[i].set_ylim([-3,3])
    if SAVEFIG:
        plt.savefig(f'figs/[20210809_compare_ML2human]_model_display_spiral_{str(ib.date.date())}.jpg', bbox_inches='tight')

NameError: name 'plt' is not defined

In [18]:
ib.estpst[0]

[[array([0.89962869, 0.89962861, 0.89962854, ..., 0.89963754, 0.89963734,
         0.89963713]),
  array([0.00702748, 0.00704301, 0.00705858, ..., 0.1120541 , 0.11225972,
         0.11246567]),
  array([0.1796875, 0.1796875, 0.1796875, ..., 0.453125 , 0.453125 ,
         0.453125 ]),
  array([1.86005512e-29, 2.11334880e-29, 2.40202245e-29, ...,
         6.50080371e-25, 5.85968487e-25, 5.28374659e-25])],
 [array([0.95873548, 0.95873547, 0.95873546, ..., 0.9587352 , 0.95873522,
         0.95873524]),
  array([7.97099341e-07, 8.08024523e-07, 8.19845033e-07, ...,
         3.09849319e-07, 3.04977164e-07, 3.00394907e-07]),
  array([0.3125  , 0.3125  , 0.3125  , ..., 0.109375, 0.109375, 0.109375]),
  array([1.26096919e-65, 1.77057783e-65, 2.48647950e-65, ...,
         1.04100355e-65, 7.43159469e-66, 5.30604748e-66])],
 [array([0.52287303, 0.52287303, 0.52287303, ..., 0.52287303, 0.52287303,
         0.52287303]),
  array([0., 0., 0., ..., 1., 1., 1.]),
  array([0.325     , 0.325     , 0.325  